In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [2]:
import csv
df = pd.read_csv('quantum_harmonic_oscillator_timed_data.csv')
print("Dataset shape ", df.shape)

Dataset shape  (700000, 201)


In [3]:
marker_1 = int(0.2 * len(df))
marker_2 = int(0.8 * len(df))

# Split into training and testing data
train_data = df.iloc[marker_1:marker_2]
test_data = pd.concat([df.iloc[:marker_1], df.iloc[marker_2:]])

# Verifying shapes
print("Shape of train_data:", train_data.shape)
print("Shape of test_data:", test_data.shape)
print()

# TIME DATA
T = df.iloc[:, 0]
time_train = T.iloc[marker_1:marker_2]
time_test = pd.concat([T.iloc[:marker_1], T.iloc[marker_2:]])
print("Shape of train time", time_train.shape)
print("Shape of test time", time_test.shape)
print()


# TRAINING DATA
feat_train = train_data.iloc[:, 1:101].values  # 1 to 101 columns are features
targ_train = train_data.iloc[:, 101:].values  # the next 100 columns are targets

# TESTING DATA
feat_test = test_data.iloc[:, 1:101].values
targ_test = test_data.iloc[:, 101:].values

# Verifying shapes
print("Shape of feat_train:", feat_train.shape)
print("Shape of targ_train:", targ_train.shape)
print("Shape of feat_test:", feat_test.shape)
print("Shape of targ_test:", targ_test.shape)

Shape of train_data: (420000, 201)
Shape of test_data: (280000, 201)

Shape of train time (420000,)
Shape of test time (280000,)

Shape of feat_train: (420000, 100)
Shape of targ_train: (420000, 100)
Shape of feat_test: (280000, 100)
Shape of targ_test: (280000, 100)


In [4]:
time_train = time_train.values.reshape(-1, 1)
time_test = time_test.values.reshape(-1,1)
print(time_train.shape)
print(time_test.shape)

(420000, 1)
(280000, 1)


In [5]:
# Convert strings to complex numbers
time_train_tensor = np.vectorize(complex)(time_train)
time_test_tensor = np.vectorize(complex)(time_test)
F_train = np.vectorize(complex)(feat_train)
T_train = np.vectorize(complex)(targ_train)
F_test = np.vectorize(complex)(feat_test)
T_test= np.vectorize(complex)(targ_test)

In [6]:
# Convert numpy arrays to PyTorch tensors
time_train_tensor = torch.tensor(time_train_tensor, dtype = torch.complex64)
time_test_tensor = torch.tensor(time_test_tensor, dtype = torch.complex64)
F_train_tensor = torch.tensor(F_train, dtype = torch.complex64)
T_train_tensor = torch.tensor(T_train, dtype = torch.complex64)
F_test_tensor = torch.tensor(F_test, dtype = torch.complex64)
T_test_tensor = torch.tensor(T_test, dtype = torch.complex64)
print(time_test_tensor.shape, time_train_tensor.shape)
print(time_test_tensor.dtype)

torch.Size([280000, 1]) torch.Size([420000, 1])
torch.complex64


In [16]:
class WeaveNet(nn.Module):
    def __init__(self):
        super(WeaveNet, self).__init__()
        self.pos_1 = nn.Linear(100, 200, dtype=torch.complex64)
        self.pos_2 = nn.Linear(200, 100, dtype=torch.complex64)

        self.time_1 = nn.Linear(1, 20, dtype=torch.complex64)
        self.time_2 = nn.Linear(20, 100, dtype=torch.complex64)  

        self.mesh_1 = nn.Linear(200, 220, dtype=torch.complex64)  
        self.mesh_2 = nn.Linear(220, 200, dtype=torch.complex64)
        
        self.pos_3 = nn.Linear(100, 200, dtype=torch.complex64)
        self.pos_4 = nn.Linear(200, 100, dtype=torch.complex64)
        
        self.time_3 = nn.Linear(1, 20, dtype=torch.complex64)
        self.time_4 = nn.Linear(20, 100, dtype=torch.complex64)
        
        self.mesh_3 = nn.Linear(200, 220, dtype=torch.complex64)
        self.mesh_4 = nn.Linear(220, 100, dtype=torch.complex64)  

    def forward(self, pos, time):
        ### Chunk 1 ###
        pos = torch.tanh(self.pos_1(pos))
        time = torch.tanh(self.time_1(time))
        time = torch.tanh(self.time_2(time))  
        combined = torch.cat((pos, time), dim=-1)
        combined = torch.tanh(self.mesh_1(combined))

        ### Chunk 2 ###
        combined = torch.tanh(self.mesh_2(combined))
        pos, time = combined[:, :100], combined[:, 100:]

        pos = torch.tanh(self.pos_3(pos))
        time = torch.tanh(self.time_3(time))
        time = torch.tanh(self.time_4(time)) 
        combined = torch.cat((pos, time), dim=-1)
        combined = torch.tanh(self.mesh_3(combined))
        
        combined = torch.tanh(self.mesh_4(combined))
        return combined


In [14]:
class CustomLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, data_preds, data_targs):
        # data loss
        diff = data_preds - data_targs
        diff_dag = torch.conj(diff)
        loss_data = torch.mean(diff*diff_dag)
        return loss_data.float()

In [17]:
#from tensorboardX import SummaryWriter
BATCH_SIZE = 20 #change this
EPOCHS = 7 #1000s of epochs
LR = 0.01

train_data = torch.utils.data.TensorDataset(F_train_tensor, time_train_tensor, T_train_tensor)
data_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

size = len(data_loader.dataset)
model = WeaveNet()
loss_fn = CustomLoss() 
optimizer = optim.Adam(model.parameters(), lr=LR)
losses = []
    
for epoch in range(EPOCHS):
    for batch, (input_data, time_col, output_data) in enumerate(data_loader):

        pred = model(input_data, time_col)
        loss = loss_fn(pred, output_data)
        losses.append(loss.item())
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()        
        
print("Done")
#i and r instead of x and y

RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x300 and 200x220)

In [ ]:
EPOCHS = list(range(1, len(losses) + 1))
plt.plot(EPOCHS, losses)
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
# Randomly select a subset of the test data
#Evaluating
model.eval()
with torch.no_grad():
    test_pred = model(F_test_tensor, time_test_tensor)
test_pred = test_pred.numpy()

num_samples = 5  # Number of samples to plot
indices = np.random.choice(F_test_tensor.shape[0], num_samples)
print(indices)

# Get the true and predicted values for these indices
true_values = T_test_tensor[indices].numpy()
predicted_values = test_pred[indices]

# Plotting the results
fig, axs = plt.subplots(num_samples, 1, figsize=(10, 5 * num_samples))

for i in range(num_samples):
    axs[i].plot(range(100), true_values[i], color='blue', label='True Values')
    axs[i].plot(range(100), predicted_values[i], color='red', label='Predicted Values')
    axs[i].plot(range(100), true_values[i].imag, color='green', label='True Values')
    axs[i].plot(range(100), predicted_values[i].imag, color='black', label='Predicted Values')
    axs[i].set_title(f'Sample {i+1}')
    axs[i].set_xlabel('Feature Index')
    axs[i].set_ylabel('Value')
    axs[i].legend()

plt.tight_layout()
plt.show()